# Purpose
predict bird class

In [9]:
import os

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import wandb
from wandb.keras import WandbCallback

import numpy as np
import math

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras import models

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# File generators

In [2]:
# train datagen
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)
    # shear_range=0.1,
    # zoom_range=0.1,
    # horizontal_flip=True
train_generator = train_datagen.flow_from_directory(
    '../data/raw/train/',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

# valid datagen
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_datagen.flow_from_directory(
    '../data/raw/valid/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# test datagen
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    '../data/raw/test/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 58388 images belonging to 400 classes.
Found 2000 images belonging to 400 classes.
Found 2000 images belonging to 400 classes.


# Model

In [3]:
base_model=keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3))

In [4]:
base_model.trainable = False

# Callbacks

## Wandb

In [3]:
wandb.init(project='bird_multiclass')

wandb: Currently logged in as: porpoising (use `wandb login --relogin` to force relogin)


## Keras weight saving

In [4]:
checkpoint_filepath = '../artifacts/w-{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'{filepath}logs/')

# Setup

In [5]:
def conv_block(filters, kernel_size, strides):
    return models.Sequential([

        layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu')])

In [6]:
model = models.Sequential([
    layers.Input(shape=(224,224,3)),

    conv_block(64, 3, 1),
    conv_block(64, 3, 1),

    layers.MaxPooling2D(2, 2, padding='same'),
    conv_block(128, 3, 1),
    conv_block(128, 3, 1),

    layers.MaxPooling2D(2, 2, padding='same'),
    conv_block(256, 3, 1),
    conv_block(256, 3, 1),
    # conv_block(256, 3, 1),

    layers.MaxPooling2D(2, 2, padding='same'),
    conv_block(512, 3, 1),
    conv_block(512, 3, 1),
    # conv_block(512, 3, 1),

    layers.MaxPooling2D(2, 2, padding='same'),
    conv_block(512, 3, 1),
    conv_block(512, 3, 1),
    # conv_block(512, 3, 2),

    layers.GlobalMaxPooling2D(),
    layers.Dense(400, activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 224, 224, 64)      2048      
                                                                 
 sequential_1 (Sequential)   (None, 224, 224, 64)      37184     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 sequential_2 (Sequential)   (None, 112, 112, 128)     74368     
                                                                 
 sequential_3 (Sequential)   (None, 112, 112, 128)     148096    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                 

In [ ]:
model.fit(train_generator,
          epochs=21,
          validation_data=val_generator,
          callbacks=[model_checkpoint_callback, WandbCallback()])

Epoch 1/21
 140/3650 [>.............................] - ETA: 10:24 - loss: 0.0129 - accuracy: 0.9969

In [13]:
model.load_weights('../artifacts/w-18-0.21.h5')
model.evaluate(test_generator, verbose=1, batch_size=32)

63/63 [==============================] - 7s 112ms/step - loss: 0.1382 - accuracy: 0.9625


[0.13815146684646606, 0.9624999761581421]

In [15]:
new_model = tf.keras.models.load_model('../artifacts/EfficientNetB4-BIRDS-0.99.h5')

In [16]:
new_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 112, 112, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 112, 112, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 113, 113, 3)  0           ['normalization[0][0]'

In [18]:
new_model.evaluate(test_generator, verbose=1, batch_size=32)

63/63 [==============================] - 13s 145ms/step - loss: 5.6723 - accuracy: 0.0935


[5.672294616699219, 0.09350000321865082]